# Keras Application 응용
- 4가지 모델의 결과를 종합하여 결과 도출

In [2]:
from glob import glob
glob('animals/*')

['animals\\bear.jpg',
 'animals\\crocodile.jpg',
 'animals\\dolphin.jpg',
 'animals\\eagle.jpg',
 'animals\\elephant.jpg',
 'animals\\hawk.jpg',
 'animals\\lion.jpg',
 'animals\\penguin.jpg',
 'animals\\shark.jpg',
 'animals\\sparrow.jpg']

In [3]:
name_list = []
for file in glob('animals/*'):
    name = file.split('\\')[1]
    name = name.split('.')[0]
    name_list.append(name)
print(name_list)

['bear', 'crocodile', 'dolphin', 'eagle', 'elephant', 'hawk', 'lion', 'penguin', 'shark', 'sparrow']


## All models and ensemble

In [4]:
from tensorflow.keras.applications.vgg16 import VGG16, decode_predictions
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet import ResNet101

import numpy as np 
from PIL import Image

In [5]:
vgg16 = VGG16()
vgg19 = VGG19()
resnet50 = ResNet50()
resnet101 = ResNet101()

In [6]:
vgg16_list, vgg19_list, resnet50_list, resnet101_list = [],[],[],[]
ensemble = []
for file in glob('animals/*'):
    img = np.array(Image.open(file).resize((224,224)))
    yhat_v16 = vgg16.predict(img.reshape(-1, 224, 224, 3))
    yhat_v19 = vgg19.predict(img.reshape(-1, 224, 224, 3))
    yhat_r50 = resnet50.predict(img.reshape(-1, 224, 224, 3))
    yhat_v100 = resnet101.predict(img.reshape(-1, 224, 224, 3))
    yhat = (yhat_v16[0]+yhat_v19[0]+yhat_r50[0]+yhat_v100[0])/4.

    vgg16_list.append(decode_predictions(yhat_v16)[0][0][1])
    vgg19_list.append(decode_predictions(yhat_v19)[0][0][1])
    resnet50_list.append(decode_predictions(yhat_r50)[0][0][1])
    resnet101_list.append(decode_predictions(yhat_v100)[0][0][1])
    ensemble.append(decode_predictions(yhat.reshape(1,-1))[0][0][1])

## 데이터 프레임으로 합치기

In [7]:
import pandas as pd
df = pd.DataFrame({
    'Original':name_list, 'VGG16':vgg16_list, 'VGG19':vgg19_list,
    'ResNet50':resnet50_list, 'ResNet101':resnet101_list, 'Ensemble':ensemble
})
df

,Original,VGG16,VGG19,ResNet50,ResNet101,Ensemble
0,bear,ice_bear,ice_bear,ice_bear,ice_bear,ice_bear
1,crocodile,agama,frilled_lizard,American_alligator,triceratops,agama
2,dolphin,Weimaraner,hammerhead,Weimaraner,Weimaraner,Weimaraner
3,eagle,kite,kite,kite,kite,kite
4,elephant,tusker,tusker,cardigan,cardigan,cardigan
5,hawk,kite,kite,kite,kite,kite
6,lion,lion,malamute,Lhasa,miniature_schnauzer,lion
7,penguin,king_penguin,king_penguin,king_penguin,king_penguin,king_penguin
8,shark,great_white_shark,great_white_shark,great_white_shark,great_white_shark,great_white_shark
9,sparrow,jay,jay,jay,jay,jay
